Spped Extraction for tuning curves (their method)

In [1]:
import numpy as np
import pickle
import math

#which rat?
dataset = np.load('python_format/moserlab_waaga_25843_2019-09-13_22-54-22_v1.npy', allow_pickle=True)
data = dataset.item()

Speed

In [2]:
Time = data['task'][1]['tracking']['t'] - data['task'][1]['tracking']['t'][0] #times scaled to start at 0
dt = np.mean(Time[2:len(Time)-1] - Time[1:len(Time)-2]) # average interval between time points
print(len(Time))
x = data['task'][1]['tracking']['x']
y = data['task'][1]['tracking']['y']

vx = np.gradient(x,dt) #could use the actual time differences, but using dt to sum
vy = np.gradient(y,dt)

V = np.sqrt((vx**2)+(vy**2))
#they smoothed the speed data, but I don't knoww if that's necessary



436879


Generate Tuning Curves

In [3]:
sig= 5**2 #Gaussian Variance (cm^2)
sigma = np.array([[sig, 0],[0,sig]]) # Covariance Matrix
speed_threshold = 3 #cm/sec using speed greater than this

speed = V
speed_idc = np.nonzero(speed>=speed_threshold)
#print(speed)


x_Pos = x[speed_idc]
y_Pos = y[speed_idc]
Time = Time[speed_idc]

print(len(Time))
print(len(x_Pos))
#build the arena
rounding = 0
dr = 10**rounding
lim = 75 #radius of the area
L = 2*lim/dr+1



X = np.linspace(-lim,lim,int(L))
Y = np.linspace(-lim,lim,int(L))
[X,Y] = np.meshgrid(X, Y)

r = np.sqrt(X**2+Y**2)
R_sq = X**2 + Y**2


outside = r > lim
[out_y,out_x] = np.where(R_sq >= lim**2)



#generate time map
Time_map = np.zeros((int(L),int(L)))

pos = np.empty(X.shape + (2,))
pos[:,:,0] = X
pos[:,:,1] = Y # create one 3d array for X and Y

from scipy.stats import multivariate_normal

for k in range(len(Time)):
    gaussian = multivariate_normal(mean=[x_Pos[k],y_Pos[k]], cov=sigma)
    g_pdf = gaussian.pdf(pos)
    #########################################################################Reshape?
    Time_map = Time_map + g_pdf
    

print(Time_map)
#generate spike maps



263087
263087
[[2.54241184e-13 6.61145053e-13 1.66954154e-12 ... 8.97581938e-17
  2.57560475e-17 7.17698681e-18]
 [8.38688045e-13 2.16379824e-12 5.41991172e-12 ... 2.99278446e-16
  8.68500411e-17 2.44954992e-17]
 [2.67845402e-12 6.85638331e-12 1.70368001e-11 ... 9.71245323e-16
  2.85289829e-16 8.15235609e-17]
 ...
 [3.82461700e-04 7.40420438e-04 1.38071988e-03 ... 2.88714477e-07
  1.14513004e-07 4.40567017e-08]
 [2.15719837e-04 4.18204845e-04 7.80989395e-04 ... 1.39042467e-07
  5.45724677e-08 2.07743256e-08]
 [1.17458822e-04 2.28019103e-04 4.26437433e-04 ... 6.51863501e-08
  2.53214279e-08 9.53933280e-09]]


Adding a second block to run easier Spike Map time

In [4]:
#get the spiketrain
from re import sub


Spike_times = data['task'][1]['spike_timestamp']  # times of very light spike
Spike_units = data['task'][1]['spike_cluster_id'] # corresponding cell that fired
Time = data['task'][1]['tracking']['t']

Time =  Time - Time[0]

Units = np.unique(Spike_units) # distinct cells that spike
N = len(Units) # number of cells

dt = np.mean(Time[2:len(Time)-1] - Time[1:len(Time)-2]) # average interval between time points
Tp = len(Time) # number of time points

ST = np.zeros((N,Tp)) # contains number of spikes for each neuron for each time bin
#print(ST.shape)
for n in range(N): # for every neuron
    u = np.where(Spike_units==Units[n]) #indexes of spikes 
    st = Spike_times[u] #times of spikes for neuron n
    st = st - st[0] #this I added because the first spikes don't begin for so long that it doesn't fit in shape Tp

    D = st//dt # D is the list of time bins for each spike of neuron n
    # in matlab they set all bins labeled 0 to bin 1, not sure if necessary, only for spikes occuring before one full dt has passed
    
    for k in range(len(D)):
        x = int(D[k])
        
        ST[n][x]=ST[n][x]+1

#print(ST)
#print(len(ST))
def sub2ind(sz, row, col):
    n_rows = sz[0]
    return [n_rows * (c-1) + r for r, c in zip(row, col)]

idc = sub2ind(Time_map.shape, out_y, out_x) #this doesn't work
#print(idc)

ST = ST[:,speed_idc] # the threshold speed Spike segments
ST = np.squeeze(ST)
#print(ST)
#print(ST.shape)

N,temp = ST.shape # N is the number of neurons

Tuning_Curves = np.zeros((int(L),int(L),N)) #empty list of tuning curves

import sys
np.set_printoptions(threshold=np.inf)
for n in range(N):

    rate_map = np.zeros((int(L),int(L)))

    st = ST[n,:] # neuron n's spike train
    #print(st)
    si = np.where(st>0)
    spike_indices = si[0]
    #print(spike_indices.shape)

    for ss in range(len(spike_indices)): #for each time the neuron fires ss is the index in spike indices 
        #print(st.shape)
        spike = spike_indices[ss]
        #print(spike)
        sp = st[spike]
        #print(sp.shape)
        x_pos = x_Pos[spike_indices[ss]] # x location spike occured
        y_pos = y_Pos[spike_indices[ss]] # y location spike occured

        for ll in range(int(sp)): #for each spike in the time bucket usually once
            gaussian = multivariate_normal(mean=[x_pos,y_pos], cov=sigma)
            g_pdf = gaussian.pdf(pos)
            #########################################################################Reshape?

            rate_map = rate_map + g_pdf

    tc = (1/dt)*(rate_map/Time_map)

    for xr in range(int(L)):
        for yr in range(int(L)):
            if (xr-lim)**2+(yr-lim)**2 > lim**2:
                tc[xr,yr]=0
    #print(tc.shape)
    #print(X[outside])
    #tc[X[outside],Y[outside]]=0
    
    #print(tc)
    #print(tc[75,75])

    Tuning_Curves[:,:,n]=tc

#print(Tuning_Curves[:,:,0])
            






Plot the tuning curves

In [5]:
# import matplotlib.pyplot as plt

# for i in range(N):
#     plt.imshow(Tuning_Curves[:,:,i], cmap='hot')
#     plt.savefig('tuning_curves/test'+str(i)+'.jpg')

Decoding Time!

In [6]:
M1_indices = [3,8,9,10,12,21,23,25,30,33,35,36,39,40,41,43,44,165,168,181,184,185,186,187,188,191,195,197,201,204,205,208,210,211,213,214,215,216,217,221,222,224]


######################################################################################################
#Need a new ST because I cropped the one earlier in the kernel,  make a function to get ST

Spike_times = data['task'][1]['spike_timestamp']  # times of very light spike
Spike_units = data['task'][1]['spike_cluster_id'] # corresponding cell that fired
Time = data['task'][1]['tracking']['t']

Time =  Time - Time[0]

Units = np.unique(Spike_units) # distinct cells that spike
N = len(Units) # number of cells

dt = np.mean(Time[2:len(Time)-1] - Time[1:len(Time)-2]) # average interval between time points
Tp = len(Time) # number of time points

ST = np.zeros((N,Tp)) # contains number of spikes for each neuron for each time bin
#print(ST.shape)
for n in range(N): # for every neuron
    u = np.where(Spike_units==Units[n]) #indexes of spikes 
    st = Spike_times[u] #times of spikes for neuron n
    st = st - st[0] #this I added because the first spikes don't begin for so long that it doesn't fit in shape Tp

    D = st//dt # D is the list of time bins for each spike of neuron n
    # in matlab they set all bins labeled 0 to bin 1, not sure if necessary, only for spikes occuring before one full dt has passed
    
    for k in range(len(D)):
        x = int(D[k])
        
        ST[n][x]=ST[n][x]+1




#frames = an array of L,L,len(Time)
frames = np.zeros((int(L),int(L),len(Time)), dtype=float)
print(frames.shape)
for t in range(len(Time)):#for each time window
    frame = np.zeros((int(L),int(L))) #frame = array of L,L
    for n in range(len(M1_indices)):#for each neuron in M1
        if ST[n,t] >0: #if there is a spike in this bin 
            for spike in range(int(ST[n,t])):
                #get the specific tuning curve for that neuron
                tc = Tuning_Curves[:,:,M1_indices[n]] 
                frame = frame + tc

    frames[:,:,t] = frame

            #for the number of times it spikes
                #add the tuning curve to 

MemoryError: Unable to allocate 74.2 GiB for an array with shape (151, 151, 436879) and data type float64

In [ ]:
import matplotlib.pyplot as plt
print(frames.shape)
print(frames[:,:,0])
plt.imshow(frames[:,:,50], cmap='hot')
#plt.savefig('tuning_curves/test'+str(i)+'.jpg')

(151, 151, 436879)
[[  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.   128.8    0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 

ValueError: Unsupported dtype

<Figure size 432x288 with 1 Axes>

Module 1 Indexes:3 8 9 10 12 21 23 25 30 33 35 36 39 40 41 43 44 165 168 181 184 185 186 187 188 191 195 197 201 204 205 208 210 211 213 214 215 216 217 221 222 224 